In [6]:
import pandas as pd
import numpy as np
import csv
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [7]:
from keras.models import Sequential
from keras.layers import Dense , Input , Dropout , LSTM , Activation , TimeDistributed , Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.utils.np_utils import to_categorical
from keras.regularizers import l1,l2
from nltk.tokenize import word_tokenize
from itertools import repeat
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import Adam

In [128]:
X_train = pd.read_excel("C:/Users/AYAN/Downloads/archive (1)/df_score.xlsx")


In [129]:
X_train=X_train[['headline','score']]

In [130]:
x_train = X_train["headline"].values

In [131]:
X_train["score"].value_counts()


 0    195454
-1      2734
 1      1812
Name: score, dtype: int64

In [132]:
x_train.shape[0]

200000

In [133]:
for i in range(len(x_train)):
    x_train[i] = str(x_train[i])

In [134]:
Y_train=X_train["score"]

In [135]:

Y_train[Y_train == -1] = "N"
Y_train[Y_train == 0] = "NE"
Y_train[Y_train == 1] = "P"



Y_train[Y_train == "N"] = 0
Y_train[Y_train == "NE"] = 1
Y_train[Y_train == "P"] = 2



In [136]:
Y_train.value_counts()

1    195454
0      2734
2      1812
Name: score, dtype: int64

In [137]:
def read_glove_vecs(glove_file):
    with open(glove_file,encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
        
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)

        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map
words_to_index , index_to_words , words_to_vec_map = read_glove_vecs("C:/Users/AYAN/Downloads/glove.6B.100d.txt/glove.6B.100d.txt")

In [138]:
len(words_to_index)

400000

In [139]:
def sentences_to_indices(X, words_to_index, max_len):
    m = X.shape[0]                                   # number of training examples
    X_indices = np.zeros((m, max_len))
    for i in range(m):                               # loop over training examples
        words = word_tokenize(X[i])
        if len(words) <= max_len:
            words.extend(repeat(0 , max_len - len(words)))
        else :
            words = words[0:max_len]
        for j in range(max_len):
            try:
                X_indices[i, j] = words_to_index[words[j]]
            except :
                X_indices[i, j] = words_to_index['unk']
    return X_indices


def Emojify_V2(input_shape, words_to_vec_map, words_to_index , emb_matrix):
    
    maxlen = input_shape[0]
    
    model = Sequential()
    model.add(Embedding(400001 , 100 , input_length = maxlen , weights = [emb_matrix] , trainable = False))
    model.add(Conv1D(filters = 100, kernel_size = 2, strides = 2 , padding='same' , activation = 'relu',kernel_regularizer = l2(0.01)))
    model.add(MaxPooling1D(pool_size = 2 , strides = 2))

    model.add(Bidirectional(LSTM(256,return_sequences = True ,kernel_regularizer=l2(0.008))))    
    model.add(Bidirectional(LSTM(256,return_sequences = False ,kernel_regularizer=l2(0.008))))
    model.add (Dense(3,kernel_regularizer = l2(0.0085)))
    
    model.add(Dense(3,activation='softmax'))
    
    return model

In [140]:
class_weight1 = dict(zip(np.unique(Y_train), class_weight.compute_class_weight('balanced', np.unique(Y_train), 
                Y_train)))

C:\Users\AYAN\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1 2], y=0         1
1         1
2         1
3         1
4         1
         ..
199995    1
199996    1
199997    1
199998    1
199999    1
Name: score, Length: 200000, dtype: object as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [141]:
class_weight1

{0: 24.384296513045598, 1: 0.3410862231863593, 2: 36.79175864606328}

In [142]:
emb_matrix = np.zeros((400001, 100))
for word, index in words_to_index.items():
        emb_matrix[index, :] = words_to_vec_map[word]

In [143]:
emb_matrix.shape

(400001, 100)

In [144]:
xtrain , xtest , ytrain , ytest = train_test_split(x_train , Y_train , test_size = 0.2 , stratify = Y_train)

In [145]:

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(ytrain)
encoded_Y = encoder.transform(ytrain)
# convert integers to dummy variables (i.e. one hot encoded)
Y = to_categorical(encoded_Y)

encoder1 = LabelEncoder()
encoder1.fit(ytest)
encoded_Y1 = encoder1.transform(ytest)
# convert integers to dummy variables (i.e. one hot encoded)
Y_test = to_categorical(encoded_Y1)




#Y = to_categorical(ytrain)
#Y_test = to_categorical(ytest.values)

In [146]:
model = Emojify_V2((256,), words_to_vec_map, words_to_index , emb_matrix)
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 256, 100)          40000100  
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 128, 100)          20100     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 64, 100)           0         
_________________________________________________________________
bidirectional_16 (Bidirectio (None, 64, 512)           731136    
_________________________________________________________________
bidirectional_17 (Bidirectio (None, 512)               1574912   
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 1539      
_________________________________________________________________
dense_15 (Dense)             (None, 3)                

In [152]:
adam=Adam(lr=0.05)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [148]:
X_train_indices = sentences_to_indices(xtrain, words_to_index, 256)
X_test_indices = sentences_to_indices(xtest, words_to_index, 256)

In [149]:
X_train_indices.shape

(160000, 256)

In [150]:
X_train_indices

array([[343465., 294037., 167678., ..., 372306., 372306., 372306.],
       [ 76672., 153292., 243876., ..., 372306., 372306., 372306.],
       [293041., 348600., 273028., ..., 372306., 372306., 372306.],
       ...,
       [ 49059., 188692.,  49256., ..., 372306., 372306., 372306.],
       [393658., 359889., 234840., ..., 372306., 372306., 372306.],
       [256861., 110156., 354594., ..., 372306., 372306., 372306.]])

In [153]:
Y_test

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [ ]:
model.fit(X_train_indices, Y, epochs = 10, batch_size = 64, class_weight=class_weight1,validation_data = (X_test_indices , Y_test) , shuffle = True)

Epoch 1/10
 165/2500 [>.............................] - ETA: 1:25:17 - loss: 7.2064 - categorical_accuracy: 0.3862

In [ ]:
def sentences_to_indices2(X, words_to_index, max_len):
    
    X_indices = np.zeros((max_len))                          
    sentence_words = [w.lower() for w in str(X).split()]    
    j = 0    
    for w in range(max_len):
            try:
                X_indices[j] = words_to_index[sentence_words[w]]
                j += 1
            except :
                
                X_indices[j] = words_to_index['unk']
                j += 1
    
    return X_indices

In [ ]:
text="Zara India reported 72% increase in its profit to Rs 82.59 crore in FY18, driven by strong sales and lower price points that helped the company acquire new customers.Zara brand-owner Inditex and the Tata Group’s retail arm Trent, which runs Zara stores in India, reported 19.4% increase in net sales to Rs 1,221.67 crore, according to Trent’s annual report released on Friday.The Company has two separate joint ventures with the Inditex group of Spain with a shareholding of 51% (Inditex) and 49% (Trent) — one for Zara and the other for Massimo Dutti stores in India. The joint venture for Zara operates 20 stores in Delhi, Mumbai, Bengaluru, Pune, Surat, Jaipur, Chandigarh, Chennai, Mohali, Hyderabad and Gurgaon.Plans are to steadily expand the presence of Zara stores in India over the next three to four years in the major cities . The primary challenge to faster expansion is the availability of high quality retail spaces which can be expected to generate reasonable sales turnover, the company said in the report.Including in the context of brand ownership and the arrangements for merchandise supply with the majority partner entirely controlling the core customer proposition with respect to the fashion, the company views its commitment to this joint venture primarily as a financial investment and consequently, it may be appropriate not to consider this as a long-term strategic investment integral to other retail operations. The joint venture for Massimo Dutti operates three stores in Mumbai and Delhi, and this joint venture entity Massimo Dutti recorded total revenues of Rs 45.75 crore."

In [ ]:

    
test_indices = sentences_to_indices2(text, words_to_index, 256)
test_indices = test_indices.reshape(1,256)
pred = model.predict_classes(test_indices)

In [ ]:
pred

In [ ]:
model_json = model.to_json()
with open("model_sentiment.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
model.save_weights("model_sentiment.h5")
print("Saved model to disk")